# Analysis
This notebook is used to analyze sound files and extract spectra, harmonics, formant, etc.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import sounddevice as sd
import scipy
import pickle
%matplotlib qt5

In [ ]:
# helpers to play/write/read audio
fs=44100

def play_audio(y):
    sd.play(y,fs)

def write_audio(y, filename):
    sf.write(filename+'.wav',y,fs)

def read_audio(filename):
    y, fs = sf.read(filename +'.wav')
    return y

def graph_signal(y, start=None, end=None):    
    plt.figure()
    x = np.arange(len(y))/fs
    plt.plot(x, y)
    plt.xlim(start, end)


def estimate_f(y, distance):
    Y = scipy.fft.rfft(y)
    Y = abs(Y)
    peaks, _ = scipy.signal.find_peaks(Y, distance=distance)
    # estimate fundamental frequency from first few peaks
    f_est = []
    for i in range(1, 6):
        f_est.append(peaks[i]/i)
    f_est = np.mean(f_est)
    return f_est


def graph_spectrum(y, distance, title="", find_peaks=True):
    Y = scipy.fft.rfft(y)
    plt.figure()
    Y = abs(Y)
    peaks, _ = scipy.signal.find_peaks(Y, distance=distance)
    # estimate fundamental frequency from first few peaks
    f_est = estimate_f(y, distance)

    plt.title(title)
    plt.plot(Y)
    if find_peaks:
        plt.plot(peaks, Y[peaks], "x")

        # also graph multiples of the fundamental frequency
        a = (np.arange(50) + 1) * f_est
        plt.plot(a, np.zeros_like(a), '2', color='red')

# convert to mono by taking left channel (arbitrary)
def to_mono(y):
    return y[:,0]


## Visualize Vowel Formants

In [12]:
distance = 300
for v in ['a', 'e', 'i', 'o', 'u']:
    y = read_audio(f"./samples/vowel/{v}")
    y = to_mono(y)
    graph_spectrum(y, distance, title=v)

From the graphs visualized above, we can see each vowel has different levels in harmonics. A few of the vowels have a noticable second formant around the 14/15th harmonic.

Note that the peaks returned by findpeaks include the 0th harmonic, which we ignore.

In [20]:
harmonic_levels = {}
num_harmonics = 40

for v in ['a', 'e', 'i', 'o', 'u']:
    y = read_audio(f"./samples/vowel/{v}")
    y = to_mono(y)
    Y = scipy.fft.rfft(y)
    Y = abs(Y)
    # Old method: find peaks to find harmonic locations
    # peaks, _ = scipy.signal.find_peaks(Y, distance=distance)
    # peaks = peaks[1:(num_harmonics + 1)]
    
    # new method: estimate fundamental freq to get harmonic locations
    f_est = estimate_f(y, distance)
    peaks = (np.arange(num_harmonics) + 1) * f_est
    peaks = np.round(peaks).astype(int)

    # to get amplitude of harmonic, take maximum amplitude within a small radius
    RADIUS = 150
    levels = [max(Y[peak - RADIUS:peak + RADIUS]) for peak in peaks]
    # levels = Y[peaks]
    harmonic_levels[v] = levels/levels[0]

with open('vowel_harmonics_40', 'wb') as f:
    pickle.dump(harmonic_levels, f)
print("Wrote vowel harmonic levels to file 'harmonic_levels'.")

print(harmonic_levels)

Wrote vowel harmonic levels to file 'harmonic_levels'.
{'a': array([1.00000000e+00, 4.42244914e-01, 2.70963303e-01, 1.44519217e-01,
       1.56575463e-01, 5.71197326e-02, 1.79931824e-01, 1.30866394e-01,
       2.07228168e-02, 3.26827928e-02, 1.44939360e-02, 5.99529124e-03,
       9.25250944e-03, 4.94949776e-02, 9.10186052e-03, 1.93492759e-03,
       2.24668081e-03, 9.40916662e-04, 6.84528764e-04, 7.46287146e-04,
       8.27972388e-04, 3.56528881e-03, 4.14686178e-03, 1.71764115e-03,
       7.89951247e-04, 1.22531336e-03, 9.45679493e-04, 1.01899164e-03,
       1.61559440e-03, 3.74731164e-03, 5.96533992e-03, 3.46427925e-03,
       5.31793711e-03, 5.45900714e-03, 1.09682206e-02, 4.25812799e-03,
       3.08313407e-03, 2.38754090e-03, 2.99942090e-03, 1.55239116e-03]), 'e': array([1.00000000e+00, 7.09075375e-01, 2.91944480e-01, 8.93113038e-02,
       8.42916607e-02, 2.53495025e-02, 1.59266422e-02, 5.83055033e-02,
       9.50779930e-02, 1.90206251e-02, 1.37401617e-02, 1.11471728e-02,
       2.

# Visualize Consonants (failed)
Starting with the /s/ phoneme

In [64]:
y = read_audio(f"./samples/consonant/s")
y = to_mono(y)

Y = scipy.fft.rfft(y)
low = 1000
high = 20000
# Y[0:low] = 0 * Y[0:low]
Y[high:] = 0 * Y[high:]
graph_spectrum(y, 300, find_peaks=False)

n = len(Y)
Y2 = np.random.normal(0, 1, n) * 100
X = np.arange(n).astype(float)

sc = np.maximum(np.zeros_like(X), (5000 - X) * (X - 20000) / (7500**2))
Y2 = Y2 * sc
plt.figure()
plt.plot(Y2)

y2 = scipy.fft.irfft(Y2, len(y))
# plt.figure()
# plt.plot(y2)
play_audio(y2)
write_audio(y2, "output/s")